In [ ]:
import google.generativeai as genai
import requests
from bs4 import BeautifulSoup
import json
from google.colab import userdata
from tqdm import tqdm
import time

# Cache pour stocker les réponses déjà obtenues
cache_file = 'response_cache.json'

# Charger le cache si disponible
def load_cache():
    try:
        with open(cache_file, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        return {}

# Sauvegarder le cache
def save_cache(cache):
    with open(cache_file, 'w') as file:
        json.dump(cache, file)

# Fonction de scraping
def scrape_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    paragraphs = [p.get_text(strip=True) for p in soup.find_all(class_="description-body")]

    # Scrape tables
    tables = []
    for table_class in ["tables-mobile", "table datasources-table table-bordered"]:
        table_elements = soup.find_all(class_=table_class)
        for table_element in table_elements:
            table = []
            rows = table_element.find_all("tr")
            headers = [th.get_text(strip=True) for th in rows[0].find_all("th")]
            id_index = headers.index("ID") if "ID" in headers else -1

            for row in rows:
                cols = row.find_all(["td", "th"])
                cols = [col.get_text(strip=True) for idx, col in enumerate(cols) if idx != id_index] if id_index != -1 else [col.get_text(strip=True) for col in cols]
                table.append(cols)
            tables.append(table)
    return paragraphs, tables

# Extraire les liens
def extract_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    container = soup.find('div', class_='matrix-container p-3')
    links = container.find_all('a', href=True)
    return [(link.text, link['href']) for link in links]

# Fonction de gestion des requêtes avec backoff
def make_request_with_backoff(api_call, max_retries=5, initial_wait=30):
    retries = 0
    wait_time = initial_wait

    while retries < max_retries:
        try:
            response = api_call()
            if hasattr(response, 'error'):
                raise RuntimeError(response.error)  # Adapter selon les erreurs spécifiques
            return response
        except Exception as e:
            if 'rate limit' in str(e).lower():  # Adaptez cette condition selon le message d'erreur
                print(f"Rate limit hit. Retrying after {wait_time} seconds...")
                time.sleep(wait_time)
                wait_time *= 2  # Backoff exponentiel
            else:
                print(f"Error: {e}")
                break
        retries += 1

    print("Max retries reached. Request failed.")
    return None

# URL principal
main_url = 'https://attack.mitre.org/'

# Extraction des liens
extracted_links = extract_links(main_url)

# Charger le cache
cache = load_cache()

# Traiter les liens en lots
batch_size = 5
gemini_key = userdata.get('key')
genai.configure(api_key=gemini_key)
model = genai.GenerativeModel(model_name="gemini-1.5-pro")

# Compteur de questions
total_questions = 0

with open('CTI_MCQ.tsv', 'a') as file:
    file.write('URL\tQuestion\tOption A\tOption B\tOption C\tOption D\tGT\tPrompt\n')

    for i in range(0, len(extracted_links), batch_size):
        batch = extracted_links[i:i + batch_size]
        for text, href in tqdm(batch, desc=f"Processing batch {i // batch_size + 1}"):
            if total_questions >= 3000:  # Vérifier si le quota de 3000 questions est atteint
                break

            url = f"https://attack.mitre.org{href}"

            # Vérifier si la réponse est dans le cache
            if url in cache:
                paragraphs, tables, response_text = cache[url]
            else:
                # Scraping
                paragraphs, tables = scrape_content(url)

                # Concatenation du contenu
                content = " ".join(paragraphs) + " " + " ".join([" ".join(row) for table in tables for row in table])

                # Si le contenu est non vide, générer les questions
                if content:
                    prompt = f"""You are a cybersecurity expert specializing in cyber threat intelligence. Given the text below, please
                    generate a maximum of 10 multiple-choice questions with four possible options each.
                    Follow these requirements:
                    1. Question Format: Each question must have four options. The options should be challenging and require
                    careful consideration. Avoid creating options that could be interpreted as correct under different circumstances.
                    2. Target Audience: The questions should be suitable for security professionals with three to five years of
                    experience in cyber threat intelligence. Avoid generic questions such as “What is the objective?”, “Which
                    operating system can be targeted?”.
                    3. Content Coverage: Aim to cover various sections of the document to ensure a comprehensive evaluation of
                    the candidate’s knowledge. Include context-specific questions that require an understanding of the document’s
                    content.
                    4. Technical Precision: Use precise terminology and concepts relevant to cyber threat intelligence. Incorporate
                    situational or scenario-based questions where applicable.
                    5. Include Technique IDs and Names: Ensure that all questions, where applicable, mention both the ID and the
                    full name of the MITRE ATT&CK pattern technique.
                    6. Premise Inclusion: Each question should include a premise indicating it pertains to MITRE ATT&CK,
                    specifying the relevant platform (Enterprise, ICS, or Mobile) where necessary.
                    7. Output Format: Return the output in TSV format (must be tab-separated) with the following columns:
                    Question, Option A, Option B, Option C, Option D, Correct Answer should be one letter (A, B, C, D).
                    Important: Only return the TSV (tab separator \\t) content as specified. Do not include any additional text or commentary outside the TSV format.
                    """ + content

                    # Appel API avec backoff
                    def api_call():
                        return model.generate_content(prompt, safety_settings="BLOCK_NONE")

                    response = make_request_with_backoff(api_call)

                    # Stocker dans le cache
                    if response:
                        cache[url] = (paragraphs, tables, response.text)
                        save_cache(cache)  # Sauvegarder le cache

                    # Traitement des réponses générées
                    if response:
                        lines = response.text.strip().split('\n')[1:]  # Ignorer l'en-tête
                        for line in lines:
                            options = line.split('\t')
                            if len(options) == 6:
                                question, option_a, option_b, option_c, option_d, correct_answer = options
                                question_prompt = f"""You are given a multiple-choice question from CTI.
                                Your task is to choose the best option among the four provided. Return your answer as a single uppercase letter: A, B, C, or D.
                                *Question:* {question}
                                *Options:*
                                A) {option_a}
                                B) {option_b}
                                C) {option_c}
                                D) {option_d}
                                *Important:* The last line of your answer should contain only the single letter corresponding to the best option, with no additional text."""

                                file.write(url + '\t' + question + '\t' + option_a + '\t' + option_b + '\t' + option_c + '\t' + option_d + '\t' + correct_answer + '\t' + question_prompt.replace('  ', '').replace('\n', ' ').replace('\t', '') + '\n')
                                total_questions += 1  # Incrémenter le compteur de questions

            # Vérifier si le quota de 3000 questions est atteint après chaque question ajoutée
            if total_questions >= 3000:
                break

        # Vérifier à nouveau après le traitement de chaque lot
        if total_questions >= 3000:
            break

        # Pause entre les lots pour éviter la limite d'API
        time.sleep(60)

print(f"TSV file created successfully with {total_questions} questions.")


Processing batch 1:  80%|████████  | 4/5 [00:53<00:11, 11.01s/it]

Error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).
Max retries reached. Request failed.


Processing batch 1: 100%|██████████| 5/5 [00:55<00:00, 11.13s/it]


Error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).
Max retries reached. Request failed.


Processing batch 2: 100%|██████████| 5/5 [00:52<00:00, 10.45s/it]

...

Error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).
Max retries reached. Request failed.


Processing batch 169:  75%|███████▌  | 3/4 [00:04<00:01,  1.55s/it]WARNING:tornado.access:429 POST /v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1014.22ms


Error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).
Max retries reached. Request failed.


Processing batch 169: 100%|██████████| 4/4 [00:06<00:00,  1.59s/it]


Error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).
Max retries reached. Request failed.
TSV file created successfully with 308 questions.
